In [134]:
# Import libraries
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [135]:
df = pd.read_csv(r"D:\كتب الكلية\Graduation_Project\Metadata_Dataset\Udacity_final.csv", encoding="cp1252")
df.head()

,Course_Name,Organization,Level,Rating,Link,About,Skills
0,Data Engineer,School of Data Science,Intermediate,4.6,https://www.udacity.com//course/data-engineer-...,Data Engineering is the foundation for the new...,"Data Modeling, Data Pipelines, Data Lakes, Spa..."
1,Data Scientist,School of Data Science,advanced,4.7,https://www.udacity.com//course/data-scientist...,"Build effective machine learning models, run d...","Machine Learning, Deep Learning, Software Engi..."
2,Data Analyst,School of Data Science,Intermediate,4.6,https://www.udacity.com//course/data-analyst-n...,"Use Python, SQL, and statistics to uncover ins...","Data Wrangling, Matplotlib, Bootstrapping, Pan..."
3,C++,School of Autonomous Systems,Intermediate,4.6,https://www.udacity.com//course/c-plus-plus-na...,Get hands-on experience by building five real-...,"Data Structures &amp; Algorithms, Memory Manag..."
4,Product Manager,School of Product Management,Beginner,4.7,https://www.udacity.com//course/product-manage...,Envision and execute the development of indust...,"Product Strategy, Product Design, Product Deve..."


In [136]:
rename_map = {
    "Course_Name": "course_name",
    "About": "description"
}

df = df.rename(columns=rename_map)


In [137]:
metadata = pd.read_csv(r"D:\كتب الكلية\Graduation_Project\Metadata_Dataset\all_courses.csv")
metadata

,id,course_name,provider,description,course_status,pricing,duration_in_hours,rating,reviews,best_of_all_time,url
0,0,CS50's Introduction to Computer Science,edX,An introduction to the intellectual enterprise...,On-Demand,Free Online Course (Audit),144.0,4.5,198,1,https://www.classcentral.com/course/computer-s...
1,1,Harvard CS50 – Full Computer Science Universit...,freeCodeCamp,Learn the basics of computer science from Harv...,On-Demand,Free Video,24.0,4.5,865,0,https://www.classcentral.com/classroom/freecod...
2,2,Introduction to Computer Science and Programmi...,edX,An introduction to computer science as a tool ...,On-Demand,Free Online Course (Audit),135.0,4.5,131,1,https://www.classcentral.com/course/computer-s...
3,3,Introduction to Electrical Engineering and Com...,MIT OpenCourseWare,This course provides an integrated introductio...,On-Demand,Free Video,17.0,4.5,49,0,https://www.classcentral.com/classroom/mit-ocw...
4,4,CS50's Computer Science for Business Professio...,edX,This is CS50’s introduction to computer scienc...,On-Demand,Free Online Course (Audit),24.0,5.0,21,0,https://www.classcentral.com/course/computer-s...
...,...,...,...,...,...,...,...,...,...,...,...
1036,1036,Mastering Design Patterns with Java,CodeSignal,CodeSignal,On-Demand,Free Certificate,0.0,0.0,36,0,https://www.classcentral.com/course/codesignal...
1037,1037,"Mastering Docker: Containers, Networking, and ...",CodeSignal,CodeSignal,On-Demand,Free Certificate,0.0,0.0,39,0,https://www.classcentral.com/course/codesignal...
1038,1038,Mastering Design Patterns with C++,CodeSignal,CodeSignal,On-Demand,Free Certificate,0.0,0.0,35,0,https://www.classcentral.com/course/codesignal...
1039,1039,AI Ethics,DataCamp,DataCamp,On-Demand,Free Trial Available,1.0,0.0,32,0,https://www.classcentral.com/course/datacamp-a...


In [138]:
metadata = pd.concat([df, metadata], ignore_index=True)

In [139]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1304 entries, 0 to 1303
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   course_name        1304 non-null   object 
 1   Organization       263 non-null    object 
 2   Level              259 non-null    object 
 3   Rating             55 non-null     float64
 4   Link               263 non-null    object 
 5   description        1265 non-null   object 
 6   Skills             248 non-null    object 
 7   id                 1041 non-null   float64
 8   provider           1041 non-null   object 
 9   course_status      1041 non-null   object 
 10  pricing            1041 non-null   object 
 11  duration_in_hours  1041 non-null   float64
 12  rating             1041 non-null   float64
 13  reviews            1041 non-null   float64
 14  best_of_all_time   1041 non-null   float64
 15  url                1041 non-null   object 
dtypes: float64(6), object(10

In [140]:
metadata['description'] = metadata['description'].fillna("")
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1304 entries, 0 to 1303
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   course_name        1304 non-null   object 
 1   Organization       263 non-null    object 
 2   Level              259 non-null    object 
 3   Rating             55 non-null     float64
 4   Link               263 non-null    object 
 5   description        1304 non-null   object 
 6   Skills             248 non-null    object 
 7   id                 1041 non-null   float64
 8   provider           1041 non-null   object 
 9   course_status      1041 non-null   object 
 10  pricing            1041 non-null   object 
 11  duration_in_hours  1041 non-null   float64
 12  rating             1041 non-null   float64
 13  reviews            1041 non-null   float64
 14  best_of_all_time   1041 non-null   float64
 15  url                1041 non-null   object 
dtypes: float64(6), object(10

In [141]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    doc = nlp(text, disable=['ner', 'pos'])
    lemmas = [token.lemma_.lower() for token in doc]
    clean_lemmas = [
        lemma for lemma in lemmas
        if lemma.isalpha() and lemma not in STOP_WORDS and lemma != '-pron-'
    ]
    return ' '.join(clean_lemmas)


In [142]:
metadata['clean_text'] = metadata['description'].apply(preprocess_text)
metadata['clean_text']

0       data engineering foundation new world big data...
1       build effective machine learning model run dat...
2       use python sql statistic uncover insight commu...
3                hand experience build real world project
4       envision execute development industry define p...
                              ...                        
1299                                           codesignal
1300                                           codesignal
1301                                           codesignal
1302                                             datacamp
1303                                             datacamp
Name: clean_text, Length: 1304, dtype: object

In [143]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer(max_features = 5000, ngram_range=(1, 2))
tfidf_matrix = vectorizer.fit_transform(metadata['clean_text'])
cosine_sim = cosine_similarity(tfidf_matrix)

In [144]:
nlp = spacy.load("en_core_web_md")

# Convert each course description into a vector
metadata['spacy_vector'] = metadata['clean_text'].apply(lambda x: nlp(str(x)).vector)

# Stack them into a matrix for similarity search
spacy_matrix = np.vstack(metadata['spacy_vector'].values)

In [145]:
from sklearn.preprocessing import normalize

tfidf_matrix_norm = normalize(tfidf_matrix)  
spacy_matrix_norm = normalize(spacy_matrix)

In [146]:

# Compute similarity separately
tfidf_sim = cosine_similarity(tfidf_matrix_norm, tfidf_matrix_norm)
spacy_sim = cosine_similarity(spacy_matrix_norm, spacy_matrix_norm)

alpha = 0.6
beta = 0.4
hybrid_sim = alpha * tfidf_sim + beta * spacy_sim

similarity_matrix = cosine_similarity(hybrid_sim, hybrid_sim)

In [152]:
def recommend_by_text(query, top_n=5):
    # TF-IDF part
    query_tfidf = vectorizer.transform([query])
    tfidf_sim = cosine_similarity(query_tfidf, tfidf_matrix_norm)

    # spaCy part
    query_vec = nlp(query).vector.reshape(1, -1)
    spacy_sim = cosine_similarity(query_vec, spacy_matrix_norm)

    # Hybrid
    alpha, beta = 0.6, 0.4
    hybrid_scores = alpha * tfidf_sim + beta * spacy_sim

    # Get top matches
    top_idx = hybrid_scores.argsort()[0][-top_n:][::-1]
    return metadata.iloc[top_idx][['course_name', 'description', 'url']]

# Example usage
recommend_by_text("AI", top_n=3)

,course_name,description,url
410,Introduction to Artificial Intelligence (AI),"Explore AI fundamentals, applications, and eth...",https://www.classcentral.com/course/introducti...
817,Introduction to Responsible AI,This is an introductory-level microlearning co...,https://www.classcentral.com/course/introducti...
27,AI Product Manager,Learn to develop AI products that deliver busi...,NaN


In [148]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(metadata['clean_text'], convert_to_tensor=True)

def recommend(course_name, top_n=5):
    idx = metadata[metadata['course_name'] == course_name].index[0]
    scores = util.cos_sim(embeddings[idx], embeddings)[0]
    top_indices = np.argsort(-scores)[1:top_n+1]
    return metadata.iloc[top_indices][['course_name', 'description', 'url']]

In [149]:
recommend("CSS Runtime Performance - Optimizing Style and Layout Calculation", top_n=3)

,course_name,description,url
134,Website Performance Optimization,"Learn how browsers convert HTML, CSS and JavaS...",NaN
535,Utility First CSS Isn't Inline Styles,"Explore utility-first CSS with Sarah Dayan, de...",https://www.classcentral.com/classroom/youtube...
474,The Future of CSS Isn't CSS - Exploring Pre-pr...,Explore next-gen CSS syntax and pre-processors...,https://www.classcentral.com/classroom/youtube...
